<h1>PROJET SANTE PUBLIQUE - Analyse des données<h1>

---



# Approche générale & Preparatifs

**Approche générale:**

Nous attaquons ici le coeur de notre projet. Nous sectionnons donc cette partie avec les analyses univariée, bivariée et multivariée


Nous importons des les packages dont nous avons besoin. A noter l'utilisation du package missingno permettant la visualisation des données manquantes

In [ ]:
import os
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn import decomposition, preprocessing
from matplotlib.collections import LineCollection

Nous changons de working directory pour simplifier import et export

In [ ]:
filepath = '/Users/efkanturedi/Corteze/openfoodfacts_clean_prod_new.csv'
data_clean = pd.read_csv(filepath)

In [ ]:
pd.options.display.float_format = "{:.1f}".format
data_clean.describe()

In [ ]:
sns.set_theme()
warnings.filterwarnings("ignore", category=FutureWarning) 

Fin des préparatifs, attaquons l'analyse!

# Analyse Univariée

In [ ]:
data_clean.columns

In [ ]:
f, axes = plt.subplots(3, 3, figsize=(20, 20))

sns.despine(left=True)

# Plot a simple distribution of the desired columns
sns.distplot(data_clean['energy-kcal_100g'], color="m",ax=axes[0,0]);
axes[0,0].set_xlim((-50,1500))
sns.distplot(data_clean['fat_100g'], color="r",ax=axes[0,1]);
sns.distplot(data_clean['saturated-fat_100g'], color="g",ax=axes[0,2]);
sns.distplot(data_clean['carbohydrates_100g'], color="g",ax=axes[1,0]);
sns.distplot(data_clean['sugars_100g'], color="g",ax=axes[1,1]);
sns.distplot(data_clean['fiber_100g'], color="g",ax=axes[1,2]);
sns.distplot(data_clean['proteins_100g'], color="g",ax=axes[2,0]);
sns.distplot(data_clean['sodium_100g'], color="g",ax=axes[2,1]);
axes[2,1].set_xlim((0,30))
sns.distplot(data_clean['nutrition-score-fr_100g'], color="g",ax=axes[2,2]);

In [ ]:
plotting = data_clean['nutriscore_grade'].value_counts(normalize=True).sort_values(ascending=True)
print(plotting)
plt.pie(plotting,autopct='%.0f',labels=plotting.index)
plt.title('Distribution des nutrigrade (en %)')
plt.axis('equal')
plt.show() # Affiche le graphique

# Analyse Bivariée

Dans cette partie, nous allons regarder les relations entre les variables quantitatives de notre database. Commencons par extraire ces données et regardons la matrice de corrélation pour nous donner une intuition des potentiels variables à privilégier. 

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_data = data_clean.select_dtypes(include=numerics)
numeric_data.head()

In [ ]:
sns.heatmap(numeric_data.corr(),linewidths=.3, cmap="YlGnBu");

Nous pouvons déduire les pairs suivantes comme étant intéressantes: 

- nutriscore_100g avec energy_kcal, fat, saturated_fat et sugars
- nutriscore_100g avec fibre et proteines


In [ ]:
fig,axes=plt.subplots(1,2, sharey=True, figsize=(10,5))
data_sample=data_clean.sample(200000)

sns.regplot(data=data_sample, ax=axes[0], x='energy-kcal_100g', y='nutrition-score-fr_100g',line_kws={"color": "red"})
axes[0].set_xlim((0,1500))
axes[0].set_ylim((-100,100))

sns.regplot(data=data_sample, ax=axes[1],x='fat_100g', y='nutrition-score-fr_100g',line_kws={"color": "red"})
axes[1].set_xlim((0,100))
axes[1].set_ylim((-100,100))

plt.show()

In [ ]:
fig,axes=plt.subplots(1,2, sharey=True, figsize=(10,5))

sns.regplot(data=data_sample, ax=axes[0], x='saturated-fat_100g', y='nutrition-score-fr_100g',line_kws={"color": "red"})
axes[0].set_xlim((0,100))
axes[0].set_ylim((-100,100))

sns.regplot(data=data_sample, ax=axes[1],x='sugars_100g', y='nutrition-score-fr_100g',line_kws={"color": "red"})
axes[1].set_xlim((0,100))
axes[1].set_ylim((-100,100))

plt.show()

In [ ]:
fig,axes=plt.subplots(1,2, sharey=True, figsize=(10,5))

sns.regplot(data=data_sample, ax=axes[0], x='fiber_100g', y='nutrition-score-fr_100g',line_kws={"color": "red"}, scatter_kws={"color": "green"})
axes[0].set_xlim((0,100))
axes[0].set_ylim((-100,100))

sns.regplot(data=data_sample, ax=axes[1],x='proteins_100g', y='nutrition-score-fr_100g',line_kws={"color": "red"}, scatter_kws={"color": "green"})
axes[1].set_xlim((0,100))
axes[1].set_ylim((-100,100))

plt.show()

Notre intuition nous dis que plus un aliment a une bonne note, plus les facteurs négatifs devraient être absents


In [ ]:
fig, axes = plt.subplots(1,2,sharex=True,sharey=True, figsize=(10,5))

sns.boxplot(x="nutriscore_grade", y="fiber_100g",ax=axes[0], data=data_clean, order=['a','b','c','d','e'])
sns.boxplot(x="nutriscore_grade", y="proteins_100g",ax=axes[1], data=data_clean, order=['a','b','c','d','e'])
axes[0].set_ylim(-10,30)
axes[1].set_ylim(-10,30)

In [ ]:
fig, axes = plt.subplots(2,2,sharex=False, figsize=(20,20))

sns.boxplot(x="nutriscore_grade", y="fat_100g",ax=axes[0,0], data=data_clean, order=['a','b','c','d','e'])
sns.boxplot(x="nutriscore_grade", y="saturated-fat_100g",ax=axes[0,1], data=data_clean, order=['a','b','c','d','e'])
sns.boxplot(x="nutriscore_grade", y="sugars_100g",ax=axes[1,0], data=data_clean, order=['a','b','c','d','e'])
sns.boxplot(x="nutriscore_grade", y="nutrition-score-fr_100g",ax=axes[1,1], data=data_clean, order=['a','b','c','d','e'])

axes[0,0].set_ylim(-10,80);
axes[0,1].set_ylim(-10,80);
axes[1,0].set_ylim(-10,80);
axes[1,1].set_ylim(-30,60);


In [ ]:
#ANOVA method
def eta_squared(x,y):
    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'moyenne_classe': yi_classe.mean()})
    SCT = sum([(yj-moyenne_y)**2 for yj in y])
    SCE = sum([c['ni']*(c['moyenne_classe']-moyenne_y)**2 for c in classes])
    return SCE/SCT
    
item_list = ['energy-kcal_100g','fat_100g', 'saturated-fat_100g','sugars_100g','fiber_100g','proteins_100g']

for item in item_list:
  print(eta_squared(data_clean['nutriscore_grade'],data_clean[item]))


# Analyse en Composantes Principales

In [ ]:

def display_circles(pcs, n_comp, pca, axis_ranks, labels=None, label_rotation=0, lims=None):
    for d1, d2 in axis_ranks: # On affiche les 2 premiers plans factoriels, donc les 4 premières composantes
        if d2 < n_comp:

            # initialisation de la figure
            fig, ax = plt.subplots(figsize=(7,6))

            # détermination des limites du graphique
            if lims is not None :
                xmin, xmax, ymin, ymax = lims
            elif pcs.shape[1] < 30 :
                xmin, xmax, ymin, ymax = -1, 1, -1, 1
            else :
                xmin, xmax, ymin, ymax = min(pcs[d1,:]), max(pcs[d1,:]), min(pcs[d2,:]), max(pcs[d2,:])

            # affichage des flèches
            # s'il y a plus de 30 flèches, on n'affiche pas le triangle à leur extrémité
            if pcs.shape[1] < 30 :
                plt.quiver(np.zeros(pcs.shape[1]), np.zeros(pcs.shape[1]),
                   pcs[d1,:], pcs[d2,:], 
                   angles='xy', scale_units='xy', scale=1, color="grey")
                # (voir la doc : https://matplotlib.org/api/_as_gen/matplotlib.pyplot.quiver.html)
            else:
                lines = [[[0,0],[x,y]] for x,y in pcs[[d1,d2]].T]
                ax.add_collection(LineCollection(lines, axes=ax, alpha=.1, color='black'))
            
            # affichage des noms des variables  
            if labels is not None:  
                for i,(x, y) in enumerate(pcs[[d1,d2]].T):
                    if x >= xmin and x <= xmax and y >= ymin and y <= ymax :
                        plt.text(x, y, labels[i], fontsize='14', ha='center', va='center', rotation=label_rotation, color="blue", alpha=0.5)
            
            # affichage du cercle
            circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
            plt.gca().add_artist(circle)

            # définition des limites du graphique
            plt.xlim(xmin, xmax)
            plt.ylim(ymin, ymax)
        
            # affichage des lignes horizontales et verticales
            plt.plot([-1, 1], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-1, 1], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            plt.xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title("Cercle des corrélations (F{} et F{})".format(d1+1, d2+1))
            plt.show(block=False)
        

def display_scree_plot(pca):
    scree = pca.explained_variance_ratio_*100
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(),c="red",marker='o')
    plt.xlabel("rang de l'axe d'inertie")
    plt.ylabel("pourcentage d'inertie")
    plt.title("Eboulis des valeurs propres")
    plt.show(block=False)

In [ ]:

# choix du nombre de composantes à calculer
n_comp = 8
X = numeric_data.values
features = numeric_data.columns

# Centrage et Reduction des données
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

# Calcul des composantes principales
pca = decomposition.PCA(n_components= n_comp)
pca.fit(X_scaled)

# Eboulis des valeurs propres
display_scree_plot(pca)

# Cercle des corrélations
pcs = pca.components_
display_circles(pcs, n_comp, pca, [(0,1),(2,3)], labels=features)

plt.show()

In [ ]:
data_clean['nutriscore_grade'].unique()